In [1]:
from azureml.core import Workspace, Experiment

# Refer to Ishmeet S reply to the question: 
# https://knowledge.udacity.com/questions/371486
# ws = Workspace.get(name="udacity-project")--> This can only work if I create a workspace named "udacity-project" in the Azure ML studio .
ws = Workspace.from_config()
experiment = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()
print(ws)

Workspace name: udacity-project
Azure region: eastus2
Subscription id: 59c23a13-e17f-4934-9c23-d4a6d567d0e0
Resource group: Machine-Learning
Workspace.create(name='udacity-project', subscription_id='59c23a13-e17f-4934-9c23-d4a6d567d0e0', resource_group='Machine-Learning')


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())


Creating a new compute cluster...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-13T00:00:11.042000+00:00', 'errors': None, 'creationTime': '2021-03-13T00:00:08.916211+00:00', 'modifiedTime': '2021-03-13T00:00:24.809721+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [3]:

%%writefile conda_dependencies.yml

dependencies:
- python=3.6.9
- scikit-learn==0.24.1
- pip:
  - xgboost==0.90
  - azureml-defaults

Writing conda_dependencies.yml


In [4]:
from azureml.core import Environment

logireg_env = Environment.from_conda_specification(name = 'logireg_env', file_path = './conda_dependencies.yml')

logireg_env.register(workspace=ws)
logireg_env.python.conda_dependencies.serialize_to_string()

'dependencies:\n- python=3.6.9\n- scikit-learn==0.24.1\n- pip:\n  - xgboost==0.90\n  - azureml-defaults\n'

since the  Sklearn Hyperparameter module was deprecated we have to use the new module for hyperparameter tuning ScriptRunConfig :
https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.scriptrunconfig?view=azure-ml-py

In [5]:
from azureml.widgets import RunDetails
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment
import os
from azureml.core.script_run_config import ScriptRunConfig
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice

# Specify parameter sampler

param_sampling = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200,500,1000),
        '--max_iter': choice(50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")


# configure and submit your training run
src = ScriptRunConfig(source_directory='./',
                      script='train.py',
                      compute_target=compute_target,
                      environment=logireg_env)

#Submit job : Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.
run = experiment.submit(src)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     policy=policy,
                                     hyperparameter_sampling=param_sampling, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=8,
                                     max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

# start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_config)

# Monitor HyperDrive runs You can monitor the progress of the runs with the following Jupyter widget
# RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")


RunId: HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8?wsid=/subscriptions/59c23a13-e17f-4934-9c23-d4a6d567d0e0/resourcegroups/Machine-Learning/workspaces/udacity-project

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-03-13T00:00:43.886648][API][INFO]Experiment created<END>\n""<START>[2021-03-13T00:00:44.378901][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-03-13T00:00:44.530369][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-03-13T00:00:45.4008114Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8?wsid=/subscriptions/59c23a13-e17f-4934

In [7]:
import joblib

# get_children_sorted_by_primary_metric: Returns a list of children sorted by their best primary metric.
# Each child in the result has run id, hyperparameters, best primary metric value and status.

print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0, reverse=False, discard_no_metric=False))

# Get your best run and save the model from that run.

# get_best_run_by_primary_metric()
# Returns the best Run, or None if no child has the primary metric.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
# get_file_names()
# Returns a list of the files that are stored in association with the run.
print("Best run file names :",best_run.get_file_names())
# log_accuracy_table()
# Logs an accuracy table to the artifact store. Maybe use it for testing.
# log_accuracy_table(name, value, description='')



[{'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_3', 'hyperparameters': None, 'best_primary_metric': 0.9168437025796662, 'status': 'Completed'}, {'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_6', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_5', 'hyperparameters': None, 'best_primary_metric': 0.9165402124430956, 'status': 'Completed'}, {'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_4', 'hyperparameters': None, 'best_primary_metric': 0.9163884673748103, 'status': 'Completed'}, {'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_7', 'hyperparameters': None, 'best_primary_metric': 0.9160849772382398, 'status': 'Completed'}, {'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_0', 'hyperparameters': None, 'best_primary_metric': 0.9160849772382398, 'status': 'Completed'}, {'run_id': 'HD_039776b5-ec1b-4338-bc2e-9bac7b700bb8_1', 'hyperparameters': None, 'best_primary_metric': 0.91593

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [9]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [24]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig?view=azure-ml-py

# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    compute_target = compute_target,
    experiment_timeout_minutes=15,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=2)

In [25]:
# Submit your automl run using Autoexperiment 

remote_run = experiment.submit(automl_config, show_output = False)
remote_run.wait_for_completion()

Running on remote.


{'runId': 'AutoML_02f8cd23-70d3-4747-9196-70f8676bacaa',
 'target': 'compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-13T00:17:39.030337Z',
 'endTimeUtc': '2021-03-13T00:43:10.877471Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"59c23a13-e17f-4934-9c23-d4a6d567d0e0","resource_group":"Machine-Learning","workspace_name":"udacity-project","region":"eastus2","compute_target":"compute-cluster","spark_service":null,"azure_service":"remote","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":null,"y_max":nul

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

# get_metrics()
# Returns the metrics
print("Best run metrics :",best_run.get_metrics())
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())


In [ ]:
best_run.get_metrics()

In [ ]:
fitted_model

In [ ]:
best_run

# Cluster clean up

compute_target.delete()